# Modelo de pabellones

### Librerías

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 10 15:23:09 2020

@author: jgonzalezh

"""

import pyodbc
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import time
from pulp import *
import matplotlib.colors as mcolors
import matplotlib.transforms as mtransforms
import matplotlib.patches as mpatches
import math
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.figure_factory import create_gantt



### Clases y funciones

In [2]:
%%time

class DATA:
    def __init__(self,archivo_dmd,archivo_equipos,instr):
        self.instrumental=instr
        x=instr.drop(instr.columns[[0, 1]], axis=1).iloc[1]
        y=instr.drop(instr.columns[[0, 1]], axis=1).iloc[0]
        self.Sub_Esp=x.unique()
        self.Esp=y.unique()
        Esp=self.Esp
        shape=instr.shape
        REST3=instr
        for i in range(2,shape[1]):
            for j in range(5,shape[0]):
                REST3.iat[j,i]=instr.fillna(0).iat[j,i]*instr.fillna(0).iat[2,i]/instr.fillna(0).iat[j,1]
        similitudes=[[0 for i in range(len(Esp))]for j in range(len(Esp))]
        columnas_esp=[[] for i in range(len(Esp))]
        for i in range(REST3.shape[1]):
            for j in range(len(Esp)):
                if REST3.iat[0,i]==Esp[j]:
                    columnas_esp[j].append(i)
        instrumentos=[[0 for x in range(5,REST3.shape[0])]for i in range(len(Esp))]
        for i in range(5,REST3.shape[0]):
            for j in range(len(Esp)):
                for z in columnas_esp[j]:
                    instrumentos[j][i-5]+=REST3.iat[i,z]
        for i in range(len(Esp)):
                for j in range(len(Esp)):
                    for z in range(len(instrumentos[0])):
                        if instrumentos[j][z]>0 and instrumentos[i][z]>0:
                            similitudes[i][j]+= round(instrumentos[i][z]+instrumentos[j][z],2)
                            similitudes[j][i]+= round(instrumentos[i][z]+instrumentos[j][z],2)
        self.instrumentos=instrumentos
        self.similitudes=similitudes
        self.Especialidades=archivo_dmd.Especialidad.unique()
        self.Cobertura=archivo_dmd.Tipo.unique() 
        a=len(self.Especialidades)
        b=len(self.Cobertura)
        self.N_Especialidad=a
        self.N_Cobertura=b
        self.Dmd_semanal=np.zeros(shape=(a,b))
        self.Bloquesmax= np.zeros(shape=(a,b))
        self.Cirujanos= np.zeros(shape=(a,b))
        for i in range(len(archivo_dmd.index)):
            for e in range(a):
                for c in range(b):
                    if archivo_dmd.iat[i,0]==self.Especialidades[e]:
                        if archivo_dmd.iat[i,1]==self.Cobertura[c]:
                            self.Dmd_semanal[e,c]=archivo_dmd.iat[i,6]
                            self.Bloquesmax[e,c]=archivo_dmd.iat[i,3]
                            self.Cirujanos[e,c]=archivo_dmd.iat[i,4]                
        dias=['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo'] 
        horas=['00:00:00','00:30:00','01:00:00','01:30:00','02:00:00','02:30:00','03:00:00','03:30:00',
            '04:00:00','04:30:00','05:00:00','05:30:00','06:00:00','06:30:00','07:00:00','07:30:00',
            '08:00:00','08:30:00','09:00:00','09:30:00','10:00:00','10:30:00','11:00:00','11:30:00',
            '12:00:00','12:30:00','13:00:00','13:30:00','14:00:00','14:30:00','15:00:00','15:30:00',
            '16:00:00','16:30:00','17:00:00','17:30:00','18:00:00','18:30:00','19:00:00','19:30:00',
            '20:00:00','20:30:00','21:00:00','21:30:00','22:00:00','22:30:00','23:00:00','23:30:00']
        FTE=1
        self.turno=archivo_equipos 
        turnos=self.turno
        fill=len(turnos.index)
        turnos['Hora Inicio']=turnos['Hora inicio Turno'].dt.hour
        turnos['Minutos Inicio']=turnos['Hora inicio Turno'].dt.minute
        turnos['Hora Fin']=turnos['Hora Fin Turno'].dt.hour
        turnos['Minutos Fin']=turnos['Hora Fin Turno'].dt.minute
        self.T=[[[0 for i in range(48)]for j in range(7)]for e in range(self.N_Especialidad)]
        for g in range(fill):
            for e in  range(self.N_Especialidad):
                if turnos.iat[g,0]==self.Especialidades[e]:
                    for h in range(7):
                        if turnos.iat[g,2]==dias[h]:
                            for j in range(int(round(2*turnos.iat[g,9]+turnos.iat[g,10]/30))):
                                if j>=int(round(2*turnos.iat[g,7]+turnos.iat[g,8]/30)) and j<=int(round(2*turnos.iat[g,9]+(turnos.iat[g,10]/30))): #revisa si tiene que ponerle -1
                                    self.T[e][h][j]+=FTE

def diccionario_bloques(bloque_ini,bloque_fin):
    diccionario=[]
    h=bloque_ini
    g=bloque_fin
    for k in range(h,g):
        for j in range (k,g):    
            largo=[]
            for u in range(k):
                largo.append(0)
            for i in range(k,g):
                if i<=j:
                    largo.append(1)
                else:
                    largo.append(0)
            for p in range(g,48):
                largo.append(0)
            diccionario.append(largo)
    return diccionario
def minmax_bloques(bloques,min_,max_):
    output=[]
    for x in range(len(bloques)):
        contar=0
        for i in range(len(bloques[x])):
            contar+=bloques[x][i]
        if contar>=min_ and contar<=max_:
            output.append(bloques[x])
    return output
def posicion(a,b):
    leng=len(b)
    x=None
    for i in range(leng):
        if a==b[i]:
            x=i
    return x
def score(e,e2, b):
    x=posicion(e,b)
    y=posicion(e2,b)
    if x!=None and y!=None:
        z=data.similitudes[x][y]
    else:
        z=0
    return z

#Transformar index a inicio y finde turno

def search_ini(list):
    for i in range(48):
        if list[i-1]==0 and list[i]==1:
            z=i
    return z
def search_fin(list):
    for i in range(48):
        if list[i-1]==1 and list[i]==0:
            z=i
    return z

def turno(lista):
    #lista=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    largo=len(lista)
    i=0
    while lista[i]==0:  
        i+=1
    inicio=i
    while lista[i]==1 or lista[i+1]==1:
        fin=i
        i+=1
    if math.modf((inicio)/2)[0] ==0:
        inicio_str=str(int(math.modf((inicio)/2)[1]))+':00:00'
    else:
        inicio_str=str(int(math.modf((inicio)/2)[1]))+':30:00'
    if math.modf((fin)/2)[0] ==0:
        fin_str=str(int(math.modf((fin)/2)[1]))+':30:00'
    else:
        fin_str=str(int(math.modf(((fin)/2)+1)[1]))+':00:00'
    return [inicio_str,fin_str]

#Colores de Gantt
def colores(largo):
    colores=['rgb(34,116,66)','rgb(7,195,80)', 'rgb(93,205,187)',
             'rgb(14,194,220)','rgb(14,122,220)', 'rgb(102,14,220)',
             'rgb(199,171,235)','rgb(229,15,103)', 'rgb(234,126,31)',
             'rgb(250,212,47)','rgb(153,213,0)', 'rgb(144,251,252)',
             'rgb(110, 0, 110)','rgb(25, 90, 40)','rgb(220, 0, 110)','rgb(110, 0, 220)',
                'rgb(220, 55, 0)','rgb(220, 110, 0)','rgb(220, 175, 0)',
                'rgb(220, 220, 110)',
                'rgb(220, 220, 0)','rgb(55, 0, 0)','rgb(220, 0, 220)', 'rgb(220, 110, 220)',
                'rgb(110, 0, 0)','rgb(220, 0, 0)','rgb(175, 0, 0)','rgb(220, 0, 110)','rgb(220, 55, 110)',
                'rgb(175, 0, 110)','rgb(220, 55, 220)','rgb(220, 110, 110)',
                'rgb(220, 175, 220)','rgb(55, 0, 110)','rgb(220, 220, 220)','rgb(55, 0, 220)',
                'rgb(220, 175, 110)','rgb(175, 0, 220)','rgb(220, 0, 220)','rgb(220, 0, 0)']
    return colores[:largo]


Wall time: 0 ns


# Lectura de datos

In [3]:
DMD=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Input_2.xlsx', sheet_name='Horas semanales')
EQ=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Horarios médicos_2.xlsx')
REST=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Instrumental.xlsx',  header=None )
Horas_esp=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Porcent Horas Subesp.xlsx',  header=0, index_col=None )


### Variables PPL

In [4]:
#Transformar a formato horas
EQ['Hora inicio Turno']=pd.to_datetime(EQ['Hora inicio Turno'], format='%H:%M:%S')
EQ['Hora Fin Turno']=pd.to_datetime(EQ['Hora Fin Turno'], format='%H:%M:%S')

#Lectura de data según calse
data=DATA(DMD,EQ,REST) 

#String especialidades
E_str=data.Especialidades
#String ley-VSC
C_str=data.Cobertura
#Especialidades númericas
E=range(data.N_Especialidad)
#Tipo ley y VSC
C=range(data.N_Cobertura)

#Tipo de pabellones (no utilizada, pero manera alternativa de solucionar primer PPL)
Tipo=[0,1,2]
#Tipo en String
Tipo_str=['Tipo 1','Tipo 2','Tipo 3']

#Pabellones numerico
P=[0,1,2,3,4,5,6,7,8,9]
#Pabellones String
P_str=['Pabellón 1','Pabellón 2','Pabellón 3','Pabellón 4','Pabellón 5','Pabellón 7','Pabellón 8','UCA 1','UCA 2','UCA 3']
#Index en string (48 medias horas del día)
horas=['00:00:00','00:30:00','01:00:00','01:30:00','02:00:00','02:30:00','03:00:00','03:30:00',
    '04:00:00','04:30:00','05:00:00','05:30:00','06:00:00','06:30:00','07:00:00','07:30:00',
    '08:00:00','08:30:00','09:00:00','09:30:00','10:00:00','10:30:00','11:00:00','11:30:00',
    '12:00:00','12:30:00','13:00:00','13:30:00','14:00:00','14:30:00','15:00:00','15:30:00',
    '16:00:00','16:30:00','17:00:00','17:30:00','18:00:00','18:30:00','19:00:00','19:30:00',
    '20:00:00','20:30:00','21:00:00','21:30:00','22:00:00','22:30:00','23:00:00','23:30:00']
#Dias numericos
dias=[0,1,2,3,4] 
#Medias horas en numeros
index = np.arange(48)
#dias en string
dias_str=['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo'] 
#Tensor de demanda semanal
Q_horas=data.Dmd_semanal 

#diccionario de bloques, todas las combinaciones posibles entre 16-42 horas (8:00 am - 21:00)
dic=diccionario_bloques(16,42)
#Eliminar todos los turnos de menos de 5 bloques de largo y de más de 16
turnos=minmax_bloques(dic,5,16)

#Largo de los diferentes turnos
T=range(len(turnos))

#Data de cirujanos por especialidad y media hora
Cirujanos=data.T

#primedio de cirujanos necesarios para cada especialidad
Avg_Cirujanos=data.Cirujanos

#mínimo de bloques necesarios por especialidad
min_sem=data.Bloquesmax

#Porcentaje de horas necesarias en cada pabellón
Horas_por_pabellon=[[[0 for e in E]for c in C]for tipo in Tipo]
for i in range(Horas_esp.shape[0]):
    for e in E:
        for tipo in Tipo:
            if Horas_esp.iat[i,0]==E_str[e] and Horas_esp.iat[i,5]==Tipo_str[tipo]:
                Horas_por_pabellon[tipo][0][e]+=Horas_esp.iat[i,3] 
                Horas_por_pabellon[tipo][1][e]+=Horas_esp.iat[i,4]

In [40]:
##########################----------Visualización de turnos por especialidad y días -------#############################
"""
for e in E:
    for c in [0]:
        print(E_str[e],' ',C_str[c],'\n')
        print('\t\t', dias_str[0],'\t',dias_str[1], dias_str[2],dias_str[3],dias_str[4])
        
        for h in index [16:42]:
            if E_str[e]!='Libre' and Avg_Cirujanos[e,c]!=0:
                print(horas[h],'\t',round(Cirujanos[e][0][h]/Avg_Cirujanos[e,c],0),'\t',Cirujanos[e][1][h]/Avg_Cirujanos[e,c],'\t',Cirujanos[e][2][h]/Avg_Cirujanos[e,c],'\t',Cirujanos[e][3][h]/Avg_Cirujanos[e,c],'\t',Cirujanos[e][4][h])
                #print(horas[h],'\t',Cirujanos[e][0][h],'\t',Cirujanos[e][1][h],'\t',Cirujanos[e][2][h],'\t',Cirujanos[e][3][h],'\t',Cirujanos[e][4][h])
                
"""                
##################### Matriz de scores  ##########################
# No se utiliza en modelo actual
"""
x=0
y=0
for e in E:
    for e2 in E[e:]:
        x+=1
        y+=score(E_str[e],E_str[e2],data.Esp)
        #print(x,E_str[e],E_str[e2],score(E_str[e],E_str[e2],data.Esp))
avg=y/x
print(avg)
puntaje=[[0 for e in E] for e2 in E]
for e in E:
    for e2 in E:
        if score(E_str[e],E_str[e2],data.Esp)>=avg:
            puntaje[e][e2]=1
            #puntaje.append((E_str[e],E_str[e2],1, score(E_str[e],E_str[e2],data.Esp)))
        else:
            puntaje[e][e2]=0
            #puntaje.append((E_str[e],E_str[e2],0,0))
            
for i in puntaje:
    print(i)
            
#data.Esp           
E_str  
"""                
' '            
        

' '

## PPL Asignación de bloques

In [6]:
%%time

#Definir problema
modelo=LpProblem('Bloques_pabellones', LpMinimize) # Se define el modelo

#Variables
A=LpVariable.dicts('Bloques_propuestos', [(e,p,t,d,c) for t in T for d in dias for p in P for e in E for c in C],lowBound=0, upBound=None,cat='Continuous')

#Función objetivo 
x=0

#Minimizar el número de bloques asignados
x+=pulp.lpSum([A[(e,p,t,d,c)] for d in dias for t in T for p in P for e in E for c in C]) 
#Penalizar entre más tarde se asigna el bloque (preferir en la mañana)
x+=pulp.lpSum([A[(e,p,t,d,c)]*h*turnos[t][h] for d in dias for t in T for p in P for e in E for c in C for h in index])*0.003
#Preferir Caundo hay mnás cirujanos contratados
x+=pulp.lpSum([(-1)*(Cirujanos[e][d][h])*turnos[t][h]*A[(e,p,t,d,c)] for d in dias for t in T for p in P for e in E for c in [0] for h in index])*0.01

#Se agregan a la función objetivo 
modelo+=x
#Restricciones

# Solo se asigna a una especialidad al mismo tiempo el pabellón
for  p in P:
    for d in dias:
        for h in index:
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for e in E for t in T for c in C])<=1 


# Se satisface la demanda semanal
for e in E:
    for c in C:
        modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for h in index for d in dias for p in P])>=Q_horas[e,c]*2

#Bloques mínimos semanales, si esq tienen demanda, ROA ley no tiene por ejemplo
for e in E:
    for c in C:
        if Q_horas[e,c]>0:
            modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for d in dias for p in P])>=min_sem[e,c]

# Número de lboques máximo en simultaneo
for d in dias:
    for e in E:
        for h in index:
            for c in C:
                if E_str[e]=='Roa':
                    modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P])<=1
                elif E_str[e]=='Maxilo':
                    modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P])<=2
                else:
                    modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P])<=4   

# Se busca que los bloques tenga cirujanos disponibles (hay un promedio de cirujanois necesarios para un bloque) para los bloques LEY . 
# La restricción muy dura e infactible, se agrega como si siempre hubiera un equipo más disponible del que hay
for d in dias:
    for e in E:
        for h in index:
            for c in [0]:
                if E_str[e]!='Libre' and Q_horas[e,c]!=0:
                    modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P])<=round(Cirujanos[e][d][h]/Avg_Cirujanos[e,c]+1,0)   

#Restriccionés particulares (Se toman por complejidad en tiempos de resolución de abordar de forma correcta la restricción de instrumental)

#No hay más de tres bloques en simultaneo de rodilla
"""
Abordada por simultaneo cadera y rodilla
for d in dias:
    for e in [1]:#con eso debería qeudar rodilla
        for h in index:
            if E_str[e]=='Rodilla' #chqueo rodilla
                modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P for c in C])<=3   
            else:
                print('error restricción rodilla')
"""
#No más de tres bloques en simultaneo cadera rodilla
for d in dias:
    for h in index:
        if E_str[1]=='Rodilla' and E_str[9]=='Cadera': #chqueo rodilla
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P for c in C for e in [1,9]])<=3   
        else:
            print('error restricción rodilla-cadera simultaneos ')
#No más de tres bloques ley cadera rodilla al día
for d in dias:
    if E_str[1]=='Rodilla' and E_str[9]=='Cadera': #chqueo rodilla
        modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for p in P for c in [0] for e in [1,9]])<=3   
    else:
        print('error restricción rodilla-cadera diarios')

#No más de 5 bloques en simultaneo EESS TyP
for d in dias:
    for h in index:
        if E_str[0]=='EESS' and E_str[3]=='TyP': #chqueo 
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P for c in C for e in [0,3]])<=5  
        else:
            print('error restricción EESS TyP simultaneos ')

#No más de 5 bloques ley EES TyP al día
for d in dias:
    if E_str[0]=='EESS' and E_str[3]=='TyP': #chequeo
        modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for p in P for c in [0] for e in [0,3]])<=5   
    else:
        print('error restricción EESS TyP diarios')
        
#No más de 1 bloques en simultaneo neuro y columna
for d in dias:
    for h in index:
        if E_str[4]=='Columna' and E_str[8]=='Neuro': #chqueo 
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P for c in C for e in [4,8]])<=1  
        else:
            print('error restricción Columna Neuro simultaneos ')

#No más de 5 bloques ley EES TyP al día
for d in dias:
    if E_str[4]=='Columna' and E_str[8]=='Neuro': #chequeo
        modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for p in P for c in C for e in [4,8]])<=2  
    else:
        print('error restricción Columna Neuro diarios')


for d in dias:
    if E_str[2]=='CPQ':# and E_str[8]=='Neuro': #chequeo
        modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for p in P for c in [0] for e in [2]])>=1  
    else: 
        print('error CPQ')



Wall time: 19min 49s


## Resolver PPL

In [7]:
%%time
modelo.solve()#pulp.PULP_CBC_CMD(maxSeconds=36000, msg=0, fracGap=0))#pulp.GLPK())#
print(LpStatus[modelo.status])
print(value(modelo.objective))

Optimal
127.69516124716412
Wall time: 1min 52s


### Pasar resultados de relajación de variables a resultados enteros


In [8]:

Result= [[[[[0 for d in dias] for t in T]for p in P ]for c in C]for e in E]
for e in E:
    for p in P:
        for t in T:
            for d in dias:
                for c in C:
                    #Se agrega la especialidad al pabellón si y solo si es superior a 50%, para que no se puedan agregar dos
                    if A[(e,p,t,d,c)].varValue>0.5:
                        Result[e][c][p][t][d]=1
                        #print('E', E_str[e],'C',C_str[c], P_str[p],'T',t,'\nTurno',turnos[t][:],'\nDia',d, 'value',Result[e][c][p][t][d])
                    else:
                        Result[e][c][p][t][d]=0
                        

### Horas por especialidad

In [9]:
#Se ocupa para los porcentajes 
# (no se penazila por el total de horas si faltan de la relajación previa, se penaliza dado lo que dió el modelo anterior)
Horas=[[0 for c in C]for e in E]
for e in E:
    for p in P:
        for t in T:
            for d in dias: 
                for c in C:
                    for h in index:
                        Horas[e][c]+=Result[e][c][p][t][d]*turnos[t][h]

## 2° PPL; Asignación de pabellones

In [10]:
%%time

import pulp




#Definir problema
pabellones=LpProblem('Pabellones_asignacion', LpMinimize) # Se define el modelo
#Variables

#Misma variable que el problema pasado, mismas dimensiones
Pab=LpVariable.dicts('Pabellones', [(e,p,t,d,c) for t in T for d in dias for p in P for e in E for c in C],lowBound=0, upBound=None,cat='Binary')

#Porcentaje, que srive para que se cumpla la mayor cantidad de veces las horas solicitadas por especilidad, pero que devuelva solución cuando es infactible
Porcent=LpVariable.dicts('Porcentaje', [(e,c)for e in E for c in C],lowBound=0, upBound=1,cat='Continuous')

#Función objetivo 
#Obtener el mayor porcentaje posible de cump´limiento de las horas de pabellones
pabellones+=pulp.lpSum([Porcent[(e,c)]*(-100) for e in E for c in C])

#Restricciones

#La nueva variable es exactamente la misma que la anterior, solo puede cambiar el pabellón que fue asignada
for e in E:
    for c in C:
        for d in dias:
            for t in T:
                pabellones+=pulp.lpSum([Pab[(e,p,t,d,c)] for p in P ])>=pulp.lpSum([Result[e][c][p][t][d] for p in P])
                pabellones+=pulp.lpSum([Pab[(e,p,t,d,c)] for p in P ])<=pulp.lpSum([Result[e][c][p][t][d] for p in P])
                
#Solo se asigna una especialidad al mismo tiempo a un pabellón                
for p in P:
    for d in dias:
        for h in index:
            pabellones+=pulp.lpSum([turnos[t][h]*Pab[(e,p,t,d,c)] for e in E for t in T for c in C])<=1 

#Se cumple el mayor porcentaje de la horas requeridas de cada pabellón
#Pabellon tipo 2 [0,1,2,3,4,6,7,8,9]
#Tipo 1 [0,1,2,3,4,6]
for e in E:
    for c in C:
        #Restricción Tipo 1
        pabellones+=pulp.lpSum([turnos[t][h]*Pab[(e,p,t,d,c)] for d in dias for t in T for p in [0,1,2,3,4,6] for h in index])>=Porcent[(e,c)]*Horas_por_pabellon[0][c][e]*Horas[e][c]
        #Restricción Tipo 2
        pabellones+=pulp.lpSum([turnos[t][h]*Pab[(e,p,t,d,c)] for d in dias for t in T for p in [0,1,2,3,4,6,7,8,9] for h in index])>=Porcent[(e,c)]*(Horas_por_pabellon[0][c][e]+Horas_por_pabellon[1][c][e])*Horas[e][c]



Wall time: 8min 55s


## Solve

In [11]:
%%time

pabellones.solve()
print(LpStatus[pabellones.status])
print(value(pabellones.objective))


Optimal
-2129.980174
Wall time: 58.6 s


In [47]:
for e in E:
    for c in C:
        print (E_str[e], C_str[c],Porcent[(e,c)].varValue)

EESS Ley 0.93487075
EESS VSC 1.0
Rodilla Ley 0.64502165
Rodilla VSC 0.98323787
CPQ Ley 0.90127828
CPQ VSC 1.0
TyP Ley 0.96661491
TyP VSC 1.0
Columna Ley 0.95022624
Columna VSC 1.0
Maxilo Ley 0.91855204
Maxilo VSC 1.0
Hombro Ley 1.0
Hombro VSC 1.0
Libre Ley 1.0
Libre VSC 1.0
Neuro Ley 1.0
Neuro VSC 1.0
Cadera Ley 1.0
Cadera VSC 1.0
Roa Ley 1.0
Roa VSC 1.0


### Pasar resultados a Tensor

In [12]:
Result= [[[[[0 for d in dias] for t in T]for p in P ]for c in C]for e in E]
for e in E:
    for p in P:
        for t in T:
            for d in dias:
                for c in C:
                    Result[e][c][p][t][d]=Pab[(e,p,t,d,c)].varValue
                    #if Pab[(e,p,t,d,c)].varValue==1:
                        #print('E', E_str[e],'C',C_str[c], P_str[p],'T',t,'\nTurno',turnos[t][:],'\nDia',d)

### Chequear resultados

In [13]:
#Horas asignadas
Horas=[[0 for c in C]for e in E]
delta_de_mhoras=[[0 for c in C]for e in E]
for e in E:
    for p in P:
        for t in T:
            for d in dias: 
                for c in C:
                    for h in index:
                        Horas[e][c]+=Result[e][c][p][t][d]*turnos[t][h]
                        
#Descomentar línea siguiente para ver resultados del modelo                        
#print('Especialidad', '\t', 'Horas modelo', '\t','Horas necesarias', '\t','Delta horas')
for e in E:
    for c in C:
        delta_de_mhoras[e][c]=Horas[e][c]/2-Q_horas[e,c]
        #Descomentar línea siguiente para ver resultados del modelo 
        #print (E_str[e] + ' '+ C_str[c], '\t',Horas[e][c]/2, '\t\t',Q_horas[e,c], '\t\t',delta_de_mhoras[e][c])
                                

In [48]:
#Horas por pabellón 
for e in E:
    for c in C:
        #print(E_str[e],C_str[c])
        for p in P:
            H_P=0
            for d in dias:
                for h in index:
                    for t in T:
                        H_P+=(Result[e][c][p][t][d]*turnos[t][h]/2)
            #if H_P!=0:
                #print('\t',P_str[p], H_P)
        

## Plots carta Gantt

In [31]:
resources=[]
colors={}
l=0
for e in E:
    for c in C: 
        resources.append(E_str[e]+' '+C_str[c])

colores2=colores(len(resources))
for e in E:
    for c in C: 
        colors[E_str[e]+' '+C_str[c]]=colores2[l]
        l+=1


In [32]:

df0=[]
df1=[]
df2=[]
df3=[]
df4=[]

for e in E:
    for p in P:
        for t in T:
            for d in dias:
                for c in C:
                    if Result[e][c][p][t][d]>0:
                        if d==0:
                            df0.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==1:
                            df1.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==2:
                            df2.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==3:
                            df3.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==4:
                            df4.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       



## Plot Gannt diarias


In [33]:

                            
fig_0 = ff.create_gantt(df0, colors=colors, index_col='Resource', show_colorbar=True, title='Día Lunes',
                        showgrid_x=True, showgrid_y=True, group_tasks=True)
fig_0.show()



In [34]:
fig_1 = ff.create_gantt(df1, colors=colors, index_col='Resource', show_colorbar=True, title='Día Martes',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_1.show()



In [35]:
fig_2 = ff.create_gantt(df2, colors=colors, index_col='Resource', show_colorbar=True, title='Día Miércoles',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_2.show()




In [36]:
fig_3 = ff.create_gantt(df3, colors=colors, index_col='Resource', show_colorbar=True, title='Día Jueves',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_3.show()



In [37]:
fig_4 = ff.create_gantt(df4, colors=colors, index_col='Resource', show_colorbar=True, title='Día Viernes',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_4.show()

# Export de datos

## Horarios Cirujanos

In [41]:
db1=[]
for e in E:
    for c in [0]:
        #print(E_str[e],' ',C_str[c],'\n')
        #print('\t\t', dias_str[0],'\t',dias_str[1], dias_str[2],dias_str[3],dias_str[4])
        
        for h in index [16:42]:
            if E_str[e]!='Libre' and Avg_Cirujanos[e,c]!=0:
                #print(horas[h],'\t',round(Cirujanos[e][0][h]/Avg_Cirujanos[e,c],0),'\t',Cirujanos[e][1][h]/Avg_Cirujanos[e,c],'\t',Cirujanos[e][2][h]/Avg_Cirujanos[e,c],'\t',Cirujanos[e][3][h]/Avg_Cirujanos[e,c],'\t',Cirujanos[e][4][h])
                #print(horas[h],'\t',Cirujanos[e][0][h],'\t',Cirujanos[e][1][h],'\t',Cirujanos[e][2][h],'\t',Cirujanos[e][3][h],'\t',Cirujanos[e][4][h])
                db1.append((E_str[e],C_str[c],horas[h], Cirujanos[e][0][h],Cirujanos[e][1][h],Cirujanos[e][2][h],Cirujanos[e][3][h],Cirujanos[e][4][h]))
                
                
df1= pd.DataFrame(db1, columns=['Especialidad', 'Cobertura', 'Mediahora del día','Lunes','Martes','Miércoles','Jueves','Viernes'])    
df1.head()
                
        

,Especialidad,Cobertura,Mediahora del día,Lunes,Martes,Miércoles,Jueves,Viernes
0,EESS,Ley,08:00:00,8,11,9,7,10
1,EESS,Ley,08:30:00,8,10,9,7,10
2,EESS,Ley,09:00:00,8,8,9,7,10
3,EESS,Ley,09:30:00,8,8,9,7,10
4,EESS,Ley,10:00:00,8,8,9,8,10


## Horas en cada pabellón

In [42]:
#Horas por pabellón 
db2=[]
for e in E:
    for c in C:
        #print(E_str[e],C_str[c])
        for p in P:
            H_P=0
            for d in dias:
                for h in index:
                    for t in T:
                        H_P+=(Result[e][c][p][t][d]*turnos[t][h]/2)
            if H_P!=0:
                db2.append((E_str[e],C_str[c],P_str[p], H_P))
                
df2= pd.DataFrame(db2, columns=['Especialidad', 'Cobertura', 'Pabellón','Horas'])    
df2.head()
        

,Especialidad,Cobertura,Pabellón,Horas
0,EESS,Ley,Pabellón 2,8.0
1,EESS,Ley,Pabellón 3,8.0
2,EESS,Ley,Pabellón 5,16.0
3,EESS,Ley,Pabellón 7,5.0
4,EESS,Ley,UCA 1,13.0


## Demanda y modelo

In [43]:
#print('Especialidad', '\t', 'Horas modelo', '\t','Horas necesarias')
db3=[]
for e in E:
    for c in C:
        #print (E_str[e] + ' '+ C_str[c], '\t',Horas[e][c]/2, '\t\t',Q_horas[e,c])
        #delta_de_mhoras[e][c]=Horas[e][c]/2-Q_horas[e,c]
        db3.append((E_str[e], C_str[c], Horas[e][c]/2, Q_horas[e,c],Horas[e][c]/2-Q_horas[e,c] ))
        
df3= pd.DataFrame(db3, columns=['Especialidad', 'Cobertura', 'Horas modelo','Horas necesarias', 'Delta horas'])    
df3.head()
        

,Especialidad,Cobertura,Horas modelo,Horas necesarias,Delta horas
0,EESS,Ley,76.0,74.0,2.0
1,EESS,VSC,16.0,15.0,1.0
2,Rodilla,Ley,55.0,53.0,2.0
3,Rodilla,VSC,31.0,29.0,2.0
4,CPQ,Ley,50.5,46.0,4.5


## Bloques


In [44]:
db4=[]
for e in E:
    for p in P:
        for t in T:
            for d in dias:
                for c in C:
                    if Result[e][c][p][t][d]==1:
                        db4.append((E_str[e],C_str[c], dias_str[d], P_str[p], turno(turnos[t])[0],turno(turnos[t])[1]))
df4= pd.DataFrame(db4, columns=['Especialidad', 'Cobertura', 'Día','Pabellón', 'Hora Inicio', 'Hora Fin'])    
df4.head()                        

,Especialidad,Cobertura,Día,Pabellón,Hora Inicio,Hora Fin
0,EESS,VSC,Viernes,Pabellón 1,13:00:00,21:00:00
1,EESS,Ley,Lunes,Pabellón 2,8:00:00,16:00:00
2,EESS,Ley,Jueves,Pabellón 3,8:00:00,16:00:00
3,EESS,VSC,Viernes,Pabellón 3,13:00:00,21:00:00
4,EESS,Ley,Miércoles,Pabellón 5,8:00:00,16:00:00


In [45]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Resultados pabellones 2.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df1.to_excel(writer, sheet_name='Horarios Cirujanos')
df2.to_excel(writer, sheet_name='Horas por pabellón')
df3.to_excel(writer, sheet_name='Detalles demanda')
df4.to_excel(writer, sheet_name='Bloques')

# Close the Pandas Excel writer and output the Excel file.
writer.save()